# IDOM - A Brief Introduction

IDOM is an easily installable and powerful Python package for:

- Making responsive user interfaces with pure Python

- Doing so with a declarative API based on React's Hooks

- And rendering those layouts on the web or in Jupyter Notebooks

This Notebook shows a basic example which is explained in more depth in IDOM's documentation where you can also find more examples.

# Where Can You Learn More?

Go to https://idom-docs.herokuapp.com/ for:

- more documentation

- installation instructions

For trouble shooting/support:

- https://github.com/idom-team/idom/issues/new/choose

In [ ]:
import idom
from idom_jupyter import display


@idom.element
def ClickCount():
    count, set_count = idom.hooks.use_state(0)

    return idom.html.button(
        {"onClick": lambda event: set_count(count + 1)},
        [f"Click count: {count}"],
    )


display(ClickCount)